In [1]:
class Temperature:
    def __init__(self, celcius):
        self.celcius = celcius

    def __format__(self, format_spec):
        if not format_spec:
            format_spec = 'c'

        if format_spec == 'c':
            return f"{self.celcius} deg C"
        else:
            return f"{self.celcius*1.8 + 32} deg F"

t = Temperature(37)
print(f"{t:f}")

98.60000000000001 deg F


In [2]:

from datetime import date
d = date(2026, 12, 29)
f"{d}"
d,'%A, %B %d, %Y'.format

(datetime.date(2026, 12, 29), <function str.format(*args, **kwargs)>)

In [3]:
import time as t

class SimpleTimer:
    def __init__(self):
        self.start = 0
        self.end = 0
        self.elapsed = 0

    def __enter__(self):
        self.start = t.time()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = t.time()
        self.elapsed = self.end - self.start

with SimpleTimer() as st:
    print("initiating heavy duty...")
    t.sleep(1)
    print("ended")

print(st.elapsed)


initiating heavy duty...
ended
1.0053610801696777


In [4]:
from importlib.metadata import requires
import time 
#8.4. Saving Memory When Creating a Large Number of
# Instances
# Problem
# Your program creates a large number (e.g., millions) of instances and uses a large
# amount of memory.eneral, the resulting memory use is comparable to that of storing
# data in a tuple. 


# 1. Standard Class (Dictionary-based)
class StandardDate:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

# 2. Optimized Class (Slotted)
class SlottedDate:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day


start_time = time.time()
slotted_instances = [SlottedDate(2023, 10, 25) for _ in range(1000)]
end_time = time.time()
print(end_time - start_time)

0.00022292137145996094


In [5]:
class Processor:
    def process(self, data):
        print(f"processing data : {data}")

class UpperCaseProcessor(Processor):
    def process(self, data):
        super().process(data.upper())

class LoggerProcessor(Processor):
    def process(self, data):
        print(f"Logging... {data}")
        super().process(data)

class FinalProcessor(UpperCaseProcessor, LoggerProcessor):
    def process(self, data):
        print("Starting final processor...")
        super().process(data)

print("MRO aka method resolution order: ")

for cl in FinalProcessor.__mro__:
    print(f" -> {cl.__name__}")

p = FinalProcessor()
print("Exec output: ")
p.process("Hellow world!")

MRO aka method resolution order: 
 -> FinalProcessor
 -> UpperCaseProcessor
 -> LoggerProcessor
 -> Processor
 -> object
Exec output: 
Starting final processor...
Logging... HELLOW WORLD!
processing data : HELLOW WORLD!


This integrated challenge combines four advanced Python object-oriented patterns from the Python Cookbook: Abstract Base Classes (8.12), Descriptors for validation (8.9), Generic Initializers (8.11), and Lazy Properties for performance (8.10).

The Optimized Geometry Engine

Difficulty: Hard

Topics: ABC, Descriptors, Lazy Properties, Generic Init

Problem Statement

You are building a high-performance geometry library. To avoid repetitive code and ensure data integrity, you need to create a framework for shapes that meets the following requirements:

The Interface (ABC): Define an abstract base class Shape that enforces the implementation of an area() method in all subclasses.

The Validator (Descriptor): Create a descriptor class PositiveNumber that ensures an attribute is a number (int or float) and is greater than zero.

The Initializer (Base Class): Create a base class AutoInit that automatically assigns positional arguments to attributes based on a class-level _fields list (eliminating boilerplate __init__).

The Optimizer (Lazy Property): Create a lazy_property descriptor that computes a method's value only once and caches it as an instance attribute.

In [6]:
from abc import ABCMeta, abstractmethod

class lazy_property:
    def __init__(self, func):
        self.func = func

    def __get__(self, instance, cls):
        if instance is None: return self
        value= self.func(instance)

        setattr(instance, self.func.__name__, value)
        return value


class PositiveNumber:
    def __init__(self, name):
        self.name =name
    def __set__(self, instance, value):
        if value <= 0:
            raise ValueError(f"{self.name} must be > 0")
        instance.__dict__[self.name] = value

class AutoInit:
    _fields = []
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError(f"Expected {len(self._fields)} args")
        for name, value in zip(self._fields, args):
            setattr(self, name, value)


class Shape(metaclass=ABCMeta):
    @abstractmethod
    def area(self):
        pass

class Rectangle(AutoInit, Shape):
    _fields = ['width', 'height']

    width = PositiveNumber('width')
    height = PositiveNumber('height')

    @lazy_property
    def area(self):
        print("Performing expensive area calc...")
        return self.width*self.height
    
try:
    rect = Rectangle(10, 5)
    print(f"width {rect.width} and height {rect.height}")

    print(f"Area: {rect.area}")

    print(f"Area used cached calculations: {rect.area}")
except Exception as e:
    print(e)

width 10 and height 5
Performing expensive area calc...
Area: 50
Area used cached calculations: 50


In [7]:
import collections.abc
class SizedString:
    def __init__(self, name, maxlen):
        self.name = name
        self.maxlen = maxlen
    
    def __set__(self,instance, value):
        if not isinstance(value, str):
            raise TypeError("Expecting a string..")

        if len(value) > self.maxlen:
            raise ValueError("Too lengthly of name")
        instance.__dict__[self.name] = value

class Profile:
    username= SizedString('username', maxlen = 15)

    def __init__(self, username):
        self.username= username
    
    @classmethod
    def from_dict(self, obj):
        return cls(data['username'])

class ReadOnlyProxy:
    def __init__(self, obj):
        self._obj = obj

    def __getattr__(self, obj):
        self._obj = obj
    
    def __getattr__(self, name):
        return getattr(self._obj, name)
    
    def __setattr__(self, name, value):
        if name == '_obj':
            super().__setattr__(name, value)
        else:
            raise AttributeError("this is a read only prof instance.")


class ProfileRegistry(collections.abc.MutableMapping):
    def __init__(self):
        self._data = {}
    
    def __getitem__(self, key): return self._data[key]
    def __delitem__(self, key): return self._data[key]
    def __len__(self): return len(self._data)
    def __iter__(self): return iter(self._data)

    def __setitem__(self, key, value):
        if not isinstance(value, Profile):
            raise TypeError("type not profile")
        self._data[key] = value

registry = ProfileRegistry()

user = Profile("haseeb")
registry["coder"] = user

safe_user = ReadOnlyProxy(user)
print(f"read only proxy: {safe_user.username}")

try:
        safe_user.username = "hacker" # Fails
except AttributeError as e:
    print(f"Proxy blocked write: {e}")

try:
    user.username = "way_too_long_of_a_name_for_this" # Fails
except ValueError as e:
    print(f"Validation blocked write: {e}")

read only proxy: haseeb
Proxy blocked write: this is a read only prof instance.
Validation blocked write: Too lengthly of name


In [17]:
class AreaVisitor:
    def visit_circle(self, circle):
        return 3.14 * circle.r**2
    
    def visit_rect(self, rect):
        return rect.w * rect.h

class Shape:
    def accept(self, visitor):
        raise NotImplementedError

class Circle(Shape):
    def __init__(self, r):
        self.r = r
    def accept(self, visitor):
        return visitor.visit_circle(self)

class Rect:
    def __init__(self, w,h):
        self.w, self.h = w, h

    def accept(self, vistor):
        return vistor.visit_rect(self)

shapes = [Circle(2), Rect(3, 4)]
visitor = AreaVisitor()
areas = []
for s in shapes:
    areas.append(s.accept(visitor))
print(areas)

[12.56, 12]


The Problem: The Smart Smart-Home Network

this problem is about building a system to manage smart-home devices. Each device can be in different states (Online/Offline), needs to be cached to prevent duplicate instances for the same hardware ID, and must be able to compare its performance against others.

Requirements:

The Cache (8.25): Create a factory function or manager that uses a weakref.WeakValueDictionary to ensure that requesting a device with the same hardware_id always returns the same instance.

The Mixin (8.18): Create a LoggableMixin that provides a log_action(message) method to any class it’s mixed with.

The State Machine (8.19): Implement a Device class that delegates its send_command() method to a state object.

In the OfflineState, send_command() raises a RuntimeError.

In the OnlineState, it prints the command.

Comparison (8.24): Use @functools.total_ordering on the Device class to allow comparison based on a signal_strength attribute.

Memory Management (8.23): Devices belong to a Hub. To prevent a reference cycle where Hub points to Device and Device points back to Hub, use weakref.ref for the device.hub reference.


In [19]:
import weakref
import functools
from time import localtime

class LoggableMixin:
    def log_act(self, message):
        t = localtime()
        ts = f"{t.tm_hour}: {t.tm_min}: {t.tm_sec}"
        print(f"[{ts}] {self.__class__.__name__} : {message}")

class DeviceState:
    @staticmethod
    def send_command(device, cmd):
        device.log_act(f"Executing '{cmd}'")

class OnlineState(DeviceState):
    @staticmethod
    def send_command(device, cmd):
        device.log_act(f"executing '{cmd}'")

class OfflineState(DeviceState):
    @staticmethod
    def send_command(device, cmd):
        raise RuntimeError(f"Device {device.hdid} is offline can't send command")

@functools.total_ordering
class SmartDevice(LoggableMixin):
    def __init__(self, hdid, sigstrength):
        self.hdid = hdid
        self.sigstrength = sigstrength
        self.state = OfflineState
        self.hub = None

    @property
    def hub(self):
        return self._hub if self._hub is not None else None

    @hub.setter
    def hub(self, value):
        self._hub = weakref.ref(value)
    
    def set_state(self, state_cls):
        self._state = state_cls

    def set_command(self, cmd):
        return self._state.send_command(self, cmd)
    
    def send_command(self, cmd):
        self._state.send_command(self, cmd)

    # for total ordering:
    def __eq__(self, other):
        return self.sigstrength == other.sigstrength
    def __lt__(self, other):
        return self.sigstrength < other.sigstrength
    def __repr__(self):
        return f"Device({self.hdid}) , Signal : {self.sigstrength}"


class DeviceManager:
    def __init__(self):
        self._cache = weakref.WeakValueDictionary()
    
    def get_device(self, device, hdid, sigstrength):
        if hdid not in self._cache:
            d = SmartDevice(hdid, sigstrength)
            self._cache[hdid] = d
            return d
        return self._cache[hdid]

class Hub:
    def __init__(self, name):
        self.name = name
        self.devices = []
    def register(self, device):
        self.devices.append(device)
        device.hub = self


manager = DeviceManager()
my_hub = Hub('MainHub')

d1 = manager.get_device("light 101", sigstrength = 22)
d2 = manager.get_device("light 101")
    print(f"Are instances identical? {d1 is d2}") # True


IndentationError: unexpected indent (361382520.py, line 85)